# Assignment 3

This project compares three feedforward neural network training algorithms: Stochastic Gradient Descent (SGD), Scaled Conjugate Gradient (SCG), and LeapFrog. Using six datasets—three for classification and three for regression—the study evaluates convergence speed, stability, and predictive accuracy. Each network has a single hidden layer, with experiments across different hidden layer sizes and hyperparameters. Performance is measured using accuracy and F1-score for classification, and MSE, RMSE, and R² for regression, alongside training time and convergence behavior. The results highlight the strengths and weaknesses of each optimizer across problems of varying complexity.

## Setup

In [ ]:
! pip3 install -r requirements.txt

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.datasets import load_iris, fetch_california_housing, fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error, r2_score, confusion_matrix, mean_absolute_error
import torch
import torch.nn as nn
import torch.optim as optim
from typing import List, Tuple, Optional, Callable
from ucimlrepo import fetch_ucirepo
import json
from datetime import datetime
from itertools import product
import time
import math
from collections import defaultdict

## Data and Pre Processing

In [ ]:
def build_preprocessing_pipeline(X, classification=True):
    numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
    categorical_features = X.select_dtypes(include=['object', 'category']).columns

    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(drop='first', handle_unknown='ignore'))
    ])

    # Combine transformers
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ]
    )

    return preprocessor

In [ ]:
def preprocess_mnist(X, y, test_size=0.2, random_state=42):
    X_norm = X / 255.0

    if len(X_norm.shape) > 2:
        X_flat = X_norm.reshape(X_norm.shape[0], -1)
    else:
        X_flat = X_norm

    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)

    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(
        X_flat, y_encoded, test_size=test_size, random_state=random_state, stratify=y_encoded
    )

    return X_train, X_test, y_train, y_test

In [ ]:
def preprocess_california_housing(X, y, skewed_features=['MedInc'], log_target=True, test_size=0.2, random_state=42):

    X_processed = X.copy()
    for col in skewed_features:
        X_processed[col] = np.log1p(X_processed[col])


    if log_target:
        y_processed = np.log1p(y)
    else:
        y_processed = y.copy()
    if X_processed.isnull().any().any():
        print("Warning: NaNs found in features after log transformation.")
    if pd.isnull(y_processed).any():
        print("Warning: NaNs found in target after log transformation.")

    preprocessor = build_preprocessing_pipeline(X_processed, classification=False)
    X_scaled = preprocessor.fit_transform(X_processed)

    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y_processed, test_size=test_size, random_state=random_state
    )

    return X_train, X_test, y_train, y_test

In [ ]:
def preprocess_data(train, val, test, scale=True, impute=False, classification=True):
    X_train, y_train = train
    X_val, y_val = val
    X_test, y_test = test

    preprocessor = build_preprocessing_pipeline(X_train, classification=classification)
    X_train_processed = preprocessor.fit_transform(X_train)
    X_val_processed = preprocessor.transform(X_val)
    X_test_processed = preprocessor.transform(X_test)

    if classification:
        le = LabelEncoder()
        y_train = le.fit_transform(y_train)
        y_val = le.transform(y_val)
        y_test = le.transform(y_test)

    return X_train_processed, X_val_processed, X_test_processed, y_train, y_val, y_test

In [ ]:
def split_train_val_test(X, y, test_size=0.15, val_fraction_of_total=0.15,
                         random_state=42, stratify=None):

    X_train_val, X_test, y_train_val, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=stratify
    )

    val_size = val_fraction_of_total / (1 - test_size)

    X_train, X_val, y_train, y_val = train_test_split(
        X_train_val, y_train_val, test_size=val_size,
        random_state=random_state, stratify=y_train_val if stratify is not None else None
    )

    return (X_train, y_train), (X_val, y_val), (X_test, y_test)

### Classification

In [ ]:
# Iris Dataset
def load_iris_data():
    iris = sns.load_dataset("iris")
    print(iris.head())

    X_iris = iris.drop("species", axis=1)
    y_iris = iris["species"]
    print(X_iris.head())
    print(y_iris.head())
    return X_iris, y_iris

In [ ]:
# Stroke Prediction Dataset
def load_stroke_data():
    stroke_data = pd.read_csv("../data/healthcare-dataset-stroke-data.csv")
    print(stroke_data.shape)
    X_stroke = stroke_data.drop("stroke", axis=1)
    y_stroke = stroke_data["stroke"]
    print(X_stroke.head())
    print(y_stroke.head())
    return X_stroke, y_stroke

In [ ]:
# MNIST Dataset
def load_mnist_subset(n_samples=10000, random_state=42):
    mnist = fetch_openml('mnist_784', version=1)
    X = pd.DataFrame(mnist.data)
    y = pd.Series(mnist.target, name="digit")
    # Sample 10k rows
    X_mnist = X.sample(n=n_samples, random_state=random_state)
    y_mnist = y.loc[X_mnist.index]
    print(X_mnist.shape)
    print(X_mnist.head())
    print(y_mnist.head())
    return X_mnist, y_mnist

### Function approx.

In [ ]:
# Synthetic Sine Wave Dataset
def generate_sine_wave_data(num_samples: int = 1000, noise_level: float = 0.1, random_state: int = 42) -> Tuple[pd.DataFrame, pd.Series]:
    rng = np.random.default_rng(random_state)
    X = np.linspace(0, 2 * np.pi, num_samples)
    y = np.sin(X) + noise_level * rng.standard_normal(num_samples)
    X_sine = pd.DataFrame(X, columns=["x"])
    y_sine = pd.Series(y, name="y")
    return X_sine, y_sine

In [ ]:
# Wine quality dataset from UCI ML Repository
def load_wine_quality_data():
    wine_quality = fetch_ucirepo(id=186) 
    
    # data (as pandas dataframes) 
    X_wine = wine_quality.data.features 
    y_wine = wine_quality.data.targets 
    print(wine_quality.metadata)
    print(wine_quality.variables)
    print(X_wine.head())
    print(y_wine.head())
    print (X_wine.shape, y_wine.shape)
    return X_wine, y_wine

In [ ]:
# California Housing Dataset
def load_california_housing_data(n_samples=10000, random_state=42):
    california = fetch_california_housing()
    X = pd.DataFrame(california.data, columns=california.feature_names)
    y = pd.Series(california.target, name="MedHouseVal")
    # Sample 10k rows
    X_california = X.sample(n=n_samples, random_state=random_state)
    y_california = y.loc[X_california.index]
    print(X_california.shape)
    print(X_california.head())
    print(y_california.head())
    return X_california, y_california

## Model

In [ ]:
class FeedforwardNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, activation_fn=nn.ReLU, dropout_p=0.0):
        super().__init__()
        self.hidden = nn.Linear(input_dim, hidden_dim)
        self.activation = activation_fn()
        self.dropout = nn.Dropout(dropout_p) if dropout_p > 0 else nn.Identity()
        self.output = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.hidden(x)
        x = self.activation(x)
        x = self.dropout(x)
        return self.output(x)

### Training Algorithms

In [ ]:
def train_sgd(model, X_train, y_train, X_val, y_val, epochs=100, lr=0.01, batch_size=32,
              classification=True, momentum=0.0, weight_decay=0.0,
              early_stopping_patience=20, min_delta=1e-4, record_times=False):
    criterion = nn.CrossEntropyLoss() if classification else nn.MSELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

    X_tr = torch.tensor(X_train, dtype=torch.float32)
    X_val_t = torch.tensor(X_val, dtype=torch.float32)
    if classification:
        y_tr = torch.tensor(y_train, dtype=torch.long)
        y_val_t = torch.tensor(y_val, dtype=torch.long)
    else:
        y_tr = torch.tensor(y_train, dtype=torch.float32)
        y_val_t = torch.tensor(y_val, dtype=torch.float32)

    ds = torch.utils.data.TensorDataset(X_tr, y_tr)
    loader = torch.utils.data.DataLoader(ds, batch_size=batch_size, shuffle=True)

    train_losses, val_losses, time_stamps = [], [], []
    best_val = float('inf')
    patience = 0
    start = time.time()

    for epoch in range(epochs):
        model.train()
        total = 0.0
        for xb, yb in loader:
            optimizer.zero_grad()
            out = model(xb)
            loss = criterion(out, yb)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            total += loss.item() * xb.size(0)
        train_loss = total / len(loader.dataset)
        train_losses.append(train_loss)

        model.eval()
        with torch.no_grad():
            val_out = model(X_val_t)
            val_loss = criterion(val_out, y_val_t).item()
        val_losses.append(val_loss)
        if record_times:
            time_stamps.append(time.time() - start)

        if val_loss + min_delta < best_val:
            best_val = val_loss
            patience = 0
        else:
            patience += 1
            if patience >= early_stopping_patience:
                break

    if record_times:
        return train_losses, val_losses, time_stamps
    return train_losses, val_losses

In [ ]:
# # Example usage of SGD - function approximation
# X_sine, y_sine = generate_sine_wave_data()
# X_train_scaled, X_test_scaled, y_train, y_test = preprocess_data(X_sine, y_sine, classification=False)
# model = FeedforwardNN(input_dim=1, hidden_dim=16, output_dim=1)
# train_losses, test_losses = train_sgd(
#     model,
#     X_train_scaled,
#     np.array(y_train).reshape(-1, 1).astype(np.float32),
#     X_test_scaled,
#     np.array(y_test).reshape(-1, 1).astype(np.float32),
#     classification=False
# )
# print("Train Losses:", train_losses)
# print("Test Losses:", test_losses)

In [ ]:
# # Example usage of SGD - Classification
# X_train_scaled, X_test_scaled, y_train, y_test = preprocess_data(X_iris, y_iris, classification=True)
# model = FeedforwardNN(input_dim=4, hidden_dim=16, output_dim=3)
# train_losses, test_losses = train_sgd(model, X_train_scaled, y_train, X_test_scaled, y_test)
# print("Train Losses:", train_losses)
# print("Test Losses:", test_losses)

In [ ]:
def train_scg(
    model: nn.Module,
    X_train: torch.Tensor,
    y_train: torch.Tensor,
    X_test: torch.Tensor,
    y_test: torch.Tensor,
    max_epochs: int = 1000,
    tolerance: float = 1e-6,
    sigma: float = 5e-5,
    lambda_init: float = 5e-7,
    verbose: bool = True,
    eval_freq: int = 10,
    grad_clip: float = 5.0,
    max_alpha: float = 1.0,
    record_times: bool = False
) -> Tuple[List[float], List[float]]:
    device = next(model.parameters()).device
    X_train, y_train = X_train.to(device), y_train.to(device)
    X_test, y_test = X_test.to(device), y_test.to(device)

    # Task detection (robust)
    if y_train.dtype in (torch.long, torch.int64) and y_train.ndim == 1:
        criterion = nn.CrossEntropyLoss()
        task_type = 'classification'
    else:
        criterion = nn.MSELoss()
        task_type = 'regression'

    def get_weights():
        return torch.cat([p.view(-1) for p in model.parameters()])

    def set_weights(w):
        idx = 0
        for p in model.parameters():
            n = p.numel()
            p.data = w[idx:idx+n].view(p.shape)
            idx += n

    def loss_and_grad(w):
        set_weights(w)
        model.zero_grad()
        out = model(X_train)
        loss = criterion(out, y_train)
        loss.backward()
        g = torch.cat([p.grad.view(-1) for p in model.parameters()])
        return loss.item(), g

    def eval_split():
        model.eval()
        with torch.no_grad():
            tr = criterion(model(X_train), y_train).item()
            te = criterion(model(X_test), y_test).item()
        model.train()
        return tr, te

    n_params = sum(p.numel() for p in model.parameters())
    w_k = get_weights()
    f_k, g_k = loss_and_grad(w_k)
    r_k = g_k.clone()
    r_k_prev = None
    p_k = -r_k.clone()
    lambda_k = lambda_init
    success = True
    k_iter = 0

    train_losses = []
    test_losses = []
    time_stamps = []
    start_time = time.time()

    tr0, te0 = eval_split()
    train_losses.append(tr0); test_losses.append(te0)
    if verbose:
        print(f"Initial - Train {tr0:.6f} Test {te0:.6f}")

    for epoch in range(max_epochs):
        if success:
            denom = torch.dot(p_k, p_k)
            if denom.abs() < 1e-18:
                if verbose: print("Direction norm too small; stopping.")
                break
            sigma_k = sigma / torch.sqrt(denom)

        # Finite difference Hessian-vector approx
        w_temp = w_k + sigma_k * p_k
        _, g_temp = loss_and_grad(w_temp)
        s_k = (g_temp - g_k) / sigma_k

        delta_k = torch.dot(p_k, s_k)
        if delta_k <= 0:
            s_k = s_k + (lambda_k - delta_k) * p_k
            delta_k = lambda_k * torch.dot(p_k, p_k)
            lambda_k *= 2

        mu_k = torch.dot(p_k, r_k)
        alpha_k = -mu_k / (delta_k + 1e-12)

        if abs(alpha_k) > max_alpha:
            alpha_k = torch.clamp(alpha_k, -max_alpha, max_alpha)

        Delta_k = -(alpha_k * mu_k + 0.5 * alpha_k * alpha_k * delta_k)

        w_new = w_k + alpha_k * p_k
        f_new, g_new = loss_and_grad(w_new)

        # NaN / Inf guard
        if not torch.isfinite(torch.tensor(f_new)):
            if verbose: print(f"Non-finite loss at epoch {epoch}; aborting this run.")
            break

        if Delta_k <= 0:
            r_ratio = -float('inf')
        else:
            r_ratio = (f_k - f_new) / Delta_k

        if r_ratio > 0:
            success = True
            w_k = w_new
            f_k = f_new
            r_k_prev = r_k.clone()
            g_k = g_new
            r_k = g_k.clone()

            if grad_clip is not None:
                g_norm = torch.norm(r_k)
                if g_norm > grad_clip:
                    r_k.mul_(grad_clip / (g_norm + 1e-12))
                    g_k = r_k  

            if r_ratio > 0.75:
                lambda_k /= 2
            elif r_ratio < 0.25:
                lambda_k *= 2
        else:
            success = False
            lambda_k *= 2  

        if success:
            if k_iter % n_params == 0 or r_k_prev is None:
                p_k = -r_k.clone()
            else:
                beta_k = torch.dot(r_k, r_k - r_k_prev) / (torch.dot(r_k_prev, r_k_prev) + 1e-12)
                p_k = -r_k + beta_k * p_k
            k_iter += 1

            grad_norm = torch.norm(r_k).item()
            if grad_norm < tolerance:
                if verbose: print(f"Converged at epoch {epoch} ||g||={grad_norm:.2e}")
                break

        if epoch % eval_freq == 0 or epoch == max_epochs - 1:
            tr, te = eval_split()
            train_losses.append(tr)
            test_losses.append(te)
            if record_times:
                time_stamps.append(time.time() - start_time)
            if verbose:
                print(f"Epoch {epoch:4d} f={tr:.6f} test={te:.6f} ||g||={torch.norm(r_k):.2e} r={r_ratio:.3f} λ={lambda_k:.2e} α={float(alpha_k):.3e}")

    if verbose and len(train_losses):
        print(f"Final - Train {train_losses[-1]:.6f} Test {test_losses[-1]:.6f}")
    if record_times:
        return train_losses, test_losses, time_stamps
    return train_losses, test_losses

**NOTE:**

*For classification tasks, the output layer is linear and we use `CrossEntropyLoss`, which applies the required softmax internally. For regression tasks, the output layer is also linear, and we use `MSELoss`.*

In [ ]:
# # Example usage of SCG - classification
# model = FeedforwardNN(input_dim=4, hidden_dim=16, output_dim=3)
# X_train_scaled, X_test_scaled, y_train, y_test = preprocess_data(X, y, classification=True)

# X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
# y_train_tensor = torch.tensor(y_train, dtype=torch.long)
# X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
# y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# train_losses, test_losses = train_scg(
#     model=model,
#     X_train=X_train_tensor,
#     y_train=y_train_tensor,
#     X_test=X_test_tensor,
#     y_test=y_test_tensor,
#     max_epochs=500,
#     tolerance=1e-5,
#     verbose=True,
#     eval_freq=1
# )
# print("Train Losses:", train_losses)
# print("Test Losses:", test_losses)

In [ ]:
# # Example usage of SCG - function approximation
# model = FeedforwardNN(input_dim=1, hidden_dim=16, output_dim=1)
# X_train_scaled, X_test_scaled, y_train, y_test = preprocess_data(X_sine, y_sine, classification=False)
# X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
# y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
# X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
# y_test_tensor = torch.tensor(y_test, dtype=torch.float32)
# train_losses, test_losses = train_scg(
#     model=model,
#     X_train=X_train_tensor,
#     y_train=y_train_tensor,
#     X_test=X_test_tensor,
#     y_test=y_test_tensor,
#     max_epochs=500,
#     tolerance=1e-5,
#     verbose=True,
#     eval_freq=1
# )
# print("Train Losses:", train_losses)
# print("Test Losses:", test_losses)

In [ ]:
def train_lfrog(
    model: nn.Module,
    X_train: torch.Tensor,
    y_train: torch.Tensor,
    X_test: torch.Tensor,
    y_test: torch.Tensor,
    loss_fn: Optional[Callable] = None,
    epochs: int = 1000,
    dt: float = 0.5,
    max_step: float = 1.0,
    convergence_tol: float = 1e-5,
    max_consecutive_decreases: int = 2,
    time_step_reduction_threshold: int = 3,
    time_step_increase_factor: float = 0.001,
    batch_size: Optional[int] = None,
    device: str = 'cpu',
    print_every: int = 100,
    early_stopping_patience: int = 50,
    min_improvement: float = 1e-6,
    record_times: bool = False
) -> Tuple[List[float], List[float]]:
    model = model.to(device)
    X_train, y_train = X_train.to(device), y_train.to(device)
    X_test, y_test = X_test.to(device), y_test.to(device)
    
    if loss_fn is None:
        if len(y_train.shape) == 1 or y_train.shape[1] == 1:
            if torch.all((y_train == 0) | (y_train == 1)):
                loss_fn = nn.BCEWithLogitsLoss()
            else:
                loss_fn = nn.MSELoss()
        else:
            loss_fn = nn.CrossEntropyLoss()
    
    train_losses = []
    test_losses = []
    time_stamps = []
    start_time = time.time()
    
    # Early stopping variables
    best_test_loss = float('inf')
    patience_counter = 0
    
    params = []
    for p in model.parameters():
        params.append(p.view(-1))
    x_k = torch.cat(params)
    n_params = len(x_k)
    
    # Compute initial gradient and velocity
    def compute_loss_and_grad():
        model.zero_grad()
        if batch_size is None:
            outputs = model(X_train)
            loss = loss_fn(outputs, y_train)
        else:
            # Mini-batch gradient
            idx = torch.randperm(len(X_train))[:batch_size]
            outputs = model(X_train[idx])
            loss = loss_fn(outputs, y_train[idx])
        
        loss.backward()
        
        grads = []
        for p in model.parameters():
            if p.grad is not None:
                grads.append(p.grad.view(-1))
            else:
                grads.append(torch.zeros_like(p.view(-1)))
        grad = torch.cat(grads)
        
        return loss.item(), grad
    
    def update_model_params(x):
        idx = 0
        for p in model.parameters():
            param_size = p.numel()
            p.data = x[idx:idx + param_size].view(p.shape)
            idx += param_size
    
    def evaluate_test():
        model.eval()
        with torch.no_grad():
            test_outputs = model(X_test)
            test_loss = loss_fn(test_outputs, y_test)
        model.train()
        return test_loss.item()
    
    train_loss, grad_k = compute_loss_and_grad()
    v_k = -0.5 * grad_k * dt  
    
    consecutive_decreases = 0
    consecutive_negative_dot_products = 0
    successful_steps = 0
    current_dt = dt
    
    print(f"Initial loss: {train_loss:.6f}, Gradient norm: {torch.norm(grad_k):.6f}")
    
    for epoch in range(epochs):
        x_k_old = x_k.clone()
        v_k_old = v_k.clone()
        grad_k_old = grad_k.clone()
        v_k_norm_old = torch.norm(v_k)
        
        # Step A: Compute step size and limit if necessary
        step_size = torch.norm(v_k) * current_dt
        if step_size > max_step:
            v_k = max_step * v_k / step_size
            step_size = max_step
        
        # Step B: Leap-frog integration
        x_k = x_k + v_k * current_dt
        update_model_params(x_k)
        
        train_loss, grad_k = compute_loss_and_grad()
        a_k = -grad_k  
        v_k = v_k + a_k * current_dt
        
        # Record losses
        test_loss = evaluate_test()
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        if record_times:
            time_stamps.append(time.time() - start_time)
        
        # Early stopping logic
        if test_loss < best_test_loss - min_improvement:
            best_test_loss = test_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= early_stopping_patience:
                print(f"Early stopping at epoch {epoch}: No improvement in test loss for {early_stopping_patience} epochs")
                print(f"Best test loss: {best_test_loss:.6f}")
                break
        
        # Step C: Check convergence
        grad_norm = torch.norm(grad_k)
        if grad_norm < convergence_tol:
            print(f"Converged at epoch {epoch}: gradient norm {grad_norm:.2e}")
            break
        
        if epoch > 0:
            dot_product = torch.dot(grad_k, grad_k_old)
            velocity_gradient_dot = torch.dot(v_k, grad_k)
            
            if dot_product <= 0:
                consecutive_negative_dot_products += 1
            else:
                consecutive_negative_dot_products = 0
                
            if velocity_gradient_dot > 0:
                consecutive_negative_dot_products += 1  
        
        # Time step reduction
        if consecutive_negative_dot_products >= time_step_reduction_threshold:
            current_dt = current_dt / 2
            current_dt = max(current_dt, 1e-6)
            x_k = (x_k + x_k_old) / 2
            v_k = (v_k + v_k_old) / 4
            update_model_params(x_k)
            consecutive_negative_dot_products = 0
            successful_steps = 0
            if print_every > 0 and epoch % print_every == 0:
                print(f"Epoch {epoch}: Reduced time step to {current_dt:.6f}")
        
        # Step D: Energy monitoring (kinetic energy check)
        v_k_norm = torch.norm(v_k)
        if v_k_norm > v_k_norm_old:
            # Kinetic energy increased - successful step
            consecutive_decreases = 0
            if step_size < max_step:
                successful_steps += 1
                growth_factor = min(1.01, 1 + successful_steps * time_step_increase_factor)
                current_dt = growth_factor * current_dt
                current_dt = max(current_dt, 1e-6)

        else:
            consecutive_decreases += 1
            successful_steps = 0
            
            x_k = (x_k + x_k_old) / 2
            update_model_params(x_k)
            
            if consecutive_decreases <= max_consecutive_decreases:
                v_k = (v_k + v_k_old) / 4
            else:
                v_k = torch.zeros_like(v_k)
                consecutive_decreases = 0
        
        if print_every > 0 and epoch % print_every == 0:
            print(f"Epoch {epoch}: Train Loss = {train_loss:.6f}, "
                  f"Test Loss = {test_loss:.6f}, Grad Norm = {grad_norm:.6f}, "
                  f"dt = {current_dt:.6f}")

    if record_times:
        return train_losses, test_losses, time_stamps
    return train_losses, test_losses

In [ ]:
# # Example usage of LFROG - classification
# model = FeedforwardNN(input_dim=4, hidden_dim=16, output_dim=3)
# X_train_scaled, X_test_scaled, y_train, y_test = preprocess_data(X, y, classification=True)

# X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
# y_train_tensor = torch.tensor(y_train, dtype=torch.long)
# X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
# y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# train_losses, test_losses = train_lfrog(
#     model=model,
#     X_train=X_train_tensor,
#     y_train=y_train_tensor,
#     X_test=X_test_tensor,
#     y_test=y_test_tensor,
#     epochs=500,
#     dt=0.005,
#     max_step=0.05,
#     early_stopping_patience=50,  # Stop if no improvement for 50 epochs
#     min_improvement=1e-6,        # Minimum improvement threshold
#     print_every=25,
#     loss_fn=nn.CrossEntropyLoss()
# )
# print("Train Losses:", train_losses)
# print("Test Losses:", test_losses)

In [ ]:
# # Example usage of LFROG - function approximation
# model = FeedforwardNN(input_dim=1, hidden_dim=16, output_dim=1)
# X_train_scaled, X_test_scaled, y_train, y_test = preprocess_data(X_sine, y_sine, classification=False)
# X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
# y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
# X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)    
# y_test_tensor = torch.tensor(y_test, dtype=torch.float32)
# train_losses, test_losses = train_lfrog(
#     model=model,
#     X_train=X_train_tensor,
#     y_train=y_train_tensor,
#     X_test=X_test_tensor,
#     y_test=y_test_tensor,   
#     epochs=500,
#     dt=0.005,
#     max_step=0.05,
#     early_stopping_patience=50,  # Stop if no improvement for 50 epochs
#     min_improvement=1e-6,        # Minimum improvement threshold
#     print_every=25,
#     loss_fn=nn.MSELoss()
# )
# print("Train Losses:", train_losses)
# print("Test Losses:", test_losses)

## Experiments

### Experiment Setup

In [ ]:
split_seed = 123
R = [42, 1337, 2025, 31415, 27182]
datasets = {}

# --- Iris (classification) ---
X_iris, y_iris = load_iris_data()
splits = split_train_val_test(X_iris, y_iris,
                              test_size=0.15, val_fraction_of_total=0.15,
                              random_state=split_seed, stratify=y_iris)
X_train_iris, X_val_iris, X_test_iris, y_train_iris, y_val_iris, y_test_iris = preprocess_data(*splits, scale=True, impute=False, classification=True)
datasets["iris"] = {
    "train": (X_train_iris, y_train_iris),
    "val":   (X_val_iris, y_val_iris),
    "test":  (X_test_iris, y_test_iris),
}

# --- Stroke Prediction (classification, needs imputation) ---
X_stroke, y_stroke = load_stroke_data()
splits = split_train_val_test(X_stroke, y_stroke,
                              test_size=0.15, val_fraction_of_total=0.15,
                              random_state=split_seed, stratify=y_stroke)
X_train_stroke, X_val_stroke, X_test_stroke, y_train_stroke, y_val_stroke, y_test_stroke = preprocess_data(*splits, scale=True, impute=True, classification=True)
datasets["stroke"] = {
    "train": (X_train_stroke, y_train_stroke),
    "val":   (X_val_stroke, y_val_stroke),
    "test":  (X_test_stroke, y_test_stroke),
}

# --- Sine Wave (regression, synthetic) ---
X_sine, y_sine = generate_sine_wave_data(num_samples=1000, noise_level=0.25, random_state=split_seed)
splits = split_train_val_test(X_sine, y_sine,
                              test_size=0.15, val_fraction_of_total=0.15,
                              random_state=split_seed, stratify=None)
X_train_sine, X_val_sine, X_test_sine, y_train_sine, y_val_sine, y_test_sine = preprocess_data(*splits, scale=True, impute=False, classification=False)
datasets["sine"] = {
    "train": (X_train_sine, y_train_sine),
    "val":   (X_val_sine, y_val_sine),
    "test":  (X_test_sine, y_test_sine),
}

# --- Wine Quality (regression) ---
X_wine, y_wine = load_wine_quality_data()
splits = split_train_val_test(X_wine, y_wine,
                              test_size=0.15, val_fraction_of_total=0.15,
                              random_state=split_seed, stratify=None)
X_train_wine, X_val_wine, X_test_wine, y_train_wine, y_val_wine, y_test_wine = preprocess_data(*splits, scale=True, impute=False, classification=False)
datasets["wine"] = {
    "train": (X_train_wine, y_train_wine),
    "val":   (X_val_wine, y_val_wine),
    "test":  (X_test_wine, y_test_wine),
}

# --- California Housing (regression) ---
X_california, y_california = load_california_housing_data(n_samples=10000, random_state=split_seed)
splits = split_train_val_test(X_california, y_california,
                              test_size=0.15, val_fraction_of_total=0.15,
                              random_state=split_seed, stratify=None)
X_train_cal, X_val_cal, X_test_cal, y_train_cal, y_val_cal, y_test_cal = preprocess_data(*splits, scale=True, impute=False, classification=False)
datasets["california"] = {
    "train": (X_train_cal, y_train_cal),
    "val":   (X_val_cal, y_val_cal),
    "test":  (X_test_cal, y_test_cal),
}

# --- MNIST (classification, pixel values 0–255 → scale to [0,1]) ---
X_mnist, y_mnist = load_mnist_subset(n_samples=10000, random_state=split_seed)
splits = split_train_val_test(X_mnist, y_mnist,
                              test_size=0.15, val_fraction_of_total=0.15,
                              random_state=split_seed, stratify=y_mnist)
(X_train, y_train), (X_val, y_val), (X_test, y_test) = splits
X_train = np.array(X_train) / 255.0
X_val   = np.array(X_val) / 255.0
X_test  = np.array(X_test) / 255.0
y_train = np.array(y_train).astype(int)
y_val   = np.array(y_val).astype(int)
y_test  = np.array(y_test).astype(int)
datasets["mnist"] = {
    "train": (X_train, y_train),
    "val":   (X_val, y_val),
    "test":  (X_test, y_test),
}

### Baseline sanity checks

In [ ]:
hidden_dim = 8  

# Sine dataset (regression)
X_train_sine, y_train_sine = datasets["sine"]["train"]
X_test_sine, y_test_sine = datasets["sine"]["test"]

model = FeedforwardNN(input_dim=X_train_sine.shape[1], hidden_dim=hidden_dim, output_dim=1, dropout_p=0.3)
torch.save(model.state_dict(), "init_sine_baseline.pth")

# SGD
train_losses, test_losses = train_sgd(
    model,
    X_train_sine,
    np.array(y_train_sine).reshape(-1, 1).astype(np.float32),
    X_test_sine,
    np.array(y_test_sine).reshape(-1, 1).astype(np.float32),
    epochs=100,
    lr=0.01,
    batch_size=32,
    classification=False,
    early_stopping_patience=15
)
print(f"Sine SGD baseline: Final train loss={train_losses[-1]:.4f}, test loss={test_losses[-1]:.4f}")

# SCG
model.load_state_dict(torch.load("init_sine_baseline.pth"))
X_train_tensor = torch.tensor(X_train_sine, dtype=torch.float32)
y_train_tensor = torch.tensor(np.array(y_train_sine).reshape(-1, 1).astype(np.float32))
X_test_tensor = torch.tensor(X_test_sine, dtype=torch.float32)
y_test_tensor = torch.tensor(np.array(y_test_sine).reshape(-1, 1).astype(np.float32))
train_losses, test_losses = train_scg(
    model,
    X_train_tensor,
    y_train_tensor,
    X_test_tensor,
    y_test_tensor,
    max_epochs=100,
    tolerance=1e-5,
    verbose=False,
    eval_freq=1
)
print(f"Sine SCG baseline: Final train loss={train_losses[-1]:.4f}, test loss={test_losses[-1]:.4f}")

# LeapFrog
model.load_state_dict(torch.load("init_sine_baseline.pth"))
train_losses, test_losses = train_lfrog(
    model=model,
    X_train=X_train_tensor,
    y_train=y_train_tensor,
    X_test=X_test_tensor,
    y_test=y_test_tensor,   
    epochs=100,
    dt=0.005,
    max_step=0.05,
    early_stopping_patience=50,  
    min_improvement=1e-6,      
    print_every=0,
    loss_fn=nn.MSELoss()
)
print(f"Sine LeapFrog baseline: Final train loss={train_losses[-1]:.4f}, test loss={test_losses[-1]:.4f}")

# Iris dataset (classification)
X_train_iris, y_train_iris = datasets["iris"]["train"]
X_test_iris, y_test_iris = datasets["iris"]["test"]
output_dim_iris = len(np.unique(y_train_iris))

model = FeedforwardNN(input_dim=X_train_iris.shape[1], hidden_dim=hidden_dim, output_dim=output_dim_iris)
torch.save(model.state_dict(), "init_iris_baseline.pth")

# SGD
train_losses, test_losses = train_sgd(
    model,
    X_train_iris,
    y_train_iris,
    X_test_iris,
    y_test_iris,
    epochs=100,
    lr=0.01,
    batch_size=32,
    classification=True
)
print(f"Iris SGD baseline: Final train loss={train_losses[-1]:.4f}, test loss={test_losses[-1]:.4f}")

# SCG
model.load_state_dict(torch.load("init_iris_baseline.pth"))
X_train_tensor = torch.tensor(X_train_iris, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_iris, dtype=torch.long)
X_test_tensor = torch.tensor(X_test_iris, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_iris, dtype=torch.long)
train_losses, test_losses = train_scg(
    model,
    X_train_tensor,
    y_train_tensor,
    X_test_tensor,
    y_test_tensor,
    max_epochs=100,
    tolerance=1e-5,
    verbose=False,
    eval_freq=1
)
print(f"Iris SCG baseline: Final train loss={train_losses[-1]:.4f}, test loss={test_losses[-1]:.4f}")

# LeapFrog
model.load_state_dict(torch.load("init_iris_baseline.pth"))
train_losses, test_losses = train_lfrog(
    model=model,
    X_train=X_train_tensor,
    y_train=y_train_tensor,
    X_test=X_test_tensor,
    y_test=y_test_tensor,
    epochs=100,
    dt=0.005,
    max_step=0.05,
    early_stopping_patience=50,  
    min_improvement=1e-6,        
    print_every=0,
    loss_fn=nn.CrossEntropyLoss()
)
print(f"Iris LeapFrog baseline: Final train loss={train_losses[-1]:.4f}, test loss={test_losses[-1]:.4f}")

### Optimal Hidden units search

In [ ]:
hidden_sizes = [2, 4, 8, 16, 32, 64, 128, 256]
n_runs = 3  

def hidden_unit_search(dataset_name, classification=True):
    X_train, y_train = datasets[dataset_name]["train"]
    X_val, y_val = datasets[dataset_name]["val"]
    output_dim = len(np.unique(y_train)) if classification else 1

    results = []
    for h in hidden_sizes:
        val_metrics = []
        accs = []
        f1s = []
        r2s = []
        mses = []
        for seed in R[:n_runs]:
            np.random.seed(seed)
            torch.manual_seed(seed)
            model = FeedforwardNN(
                input_dim=X_train.shape[1],
                hidden_dim=h,
                output_dim=output_dim
            )
            # For regression, ensure targets are float32 and shaped correctly
            if classification:
                y_train_run = y_train
                y_val_run = y_val
            else:
                y_train_run = np.array(y_train).reshape(-1, 1).astype(np.float32)
                y_val_run = np.array(y_val).reshape(-1, 1).astype(np.float32)
            _, val_losses = train_sgd(
                model,
                X_train,
                y_train_run,
                X_val,
                y_val_run,
                epochs=100,
                lr=0.01,
                batch_size=32,
                classification=classification
            )
            val_metrics.append(val_losses[-1])

            model.eval()
            X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
            with torch.no_grad():
                outputs = model(X_val_tensor)

                if classification:
                    preds = torch.argmax(outputs, dim=1).cpu().numpy()
                    accs.append(accuracy_score(y_val, preds))
                    f1s.append(f1_score(y_val, preds, average="weighted"))
                else:
                    preds = outputs.cpu().numpy().flatten()
                    mses.append(mean_squared_error(y_val_run.flatten(), preds))
                    r2s.append(r2_score(y_val_run.flatten(), preds))

        result = {
            "hidden_dim": h,
            "mean_val": float(np.mean(val_metrics)),
            "std_val": float(np.std(val_metrics)),
            "all_val": [float(x) for x in val_metrics],
        }

        if classification:
            result.update({
                "mean_acc": float(np.mean(accs)),
                "std_acc": float(np.std(accs)),
                "all_acc": accs,
                "mean_f1": float(np.mean(f1s)),
                "std_f1": float(np.std(f1s)),
                "all_f1": f1s,
            })
        else:
            result.update({
                "mean_mse": float(np.mean(mses)),
                "std_mse": float(np.std(mses)),
                "all_mse": mses,
                "mean_r2": float(np.mean(r2s)),
                "std_r2": float(np.std(r2s)),
                "all_r2": r2s,
            })

        results.append(result)

    return results

def select_best_hidden(results, task="classification", metric=None, n_runs=3, relative_rule=False, relative_pct=0.01):

    if metric is None:
        metric = "val" if task == "classification" else "mse"

    key = f"mean_{metric}"

    if relative_rule:
        if metric in ["val", "mse"]:  
            best = min(results, key=lambda x: x[key])
            threshold = best[key] * (1 + relative_pct)
            candidates = [r for r in results if r[key] <= threshold]
        else: 
            best = max(results, key=lambda x: x[key])
            threshold = best[key] * (1 - relative_pct)
            candidates = [r for r in results if r[key] >= threshold]
        se_best = None  
    else:
        if metric in ["val", "mse"]:  
            best = min(results, key=lambda x: x[key])
            se_best = best[f"std_{metric}"] / np.sqrt(n_runs)
            threshold = best[key] + se_best
            candidates = [r for r in results if r[key] <= threshold]
        else:  
            best = max(results, key=lambda x: x[key])
            se_best = best[f"std_{metric}"] / np.sqrt(n_runs)
            threshold = best[key] - se_best
            candidates = [r for r in results if r[key] >= threshold]

    parsimonious = min(candidates, key=lambda x: x["hidden_dim"])
    return parsimonious, best, se_best, threshold


In [ ]:
search_results = {}

# Sine (regression)
sine_results = hidden_unit_search("sine", classification=False)
parsimonious, best, se_best, threshold = select_best_hidden(sine_results, task="regression", metric="mse", n_runs=n_runs, relative_rule=True)
search_results["sine"] = {
    "results": sine_results,
    "selected": parsimonious,
    "best": best,
    "se_best": se_best,
    "threshold": threshold
}
print(f"Sine: Selected hidden units = {parsimonious['hidden_dim']} (mean val loss: {parsimonious['mean_val']:.4f})")

# Iris (classification)
iris_results = hidden_unit_search("iris", classification=True)
parsimonious, best, se_best, threshold = select_best_hidden(iris_results, task="classification", metric="val", n_runs=n_runs, relative_rule=True)
search_results["iris"] = {
    "results": iris_results,
    "selected": parsimonious,
    "best": best,
    "se_best": se_best,
    "threshold": threshold
}
print(f"Iris: Selected hidden units = {parsimonious['hidden_dim']} (mean val loss: {parsimonious['mean_val']:.4f})")

# Stroke (classification)
stroke_results = hidden_unit_search("stroke", classification=True)
parsimonious, best, se_best, threshold = select_best_hidden(stroke_results, task="classification", metric="val", n_runs=n_runs, relative_rule=True)
search_results["stroke"] = {
    "results": stroke_results,
    "selected": parsimonious,
    "best": best,
    "se_best": se_best,
    "threshold": threshold
}
print(f"Stroke: Selected hidden units = {parsimonious['hidden_dim']} (mean val loss: {parsimonious['mean_val']:.4f})")

# Wine (regression)
wine_results = hidden_unit_search("wine", classification=False)
parsimonious, best, se_best, threshold = select_best_hidden(wine_results, task="regression", metric="mse", n_runs=n_runs, relative_rule=True)
search_results["wine"] = {
    "results": wine_results,
    "selected": parsimonious,
    "best": best,
    "se_best": se_best,
    "threshold": threshold
}
print(f"Wine: Selected hidden units = {parsimonious['hidden_dim']} (mean val loss: {parsimonious['mean_val']:.4f})")

# California Housing (regression)
california_results = hidden_unit_search("california", classification=False)
parsimonious, best, se_best, threshold = select_best_hidden(california_results, task="regression", metric="mse", n_runs=n_runs, relative_rule=True)
search_results["california"] = {
    "results": california_results,
    "selected": parsimonious,
    "best": best,
    "se_best": se_best,
    "threshold": threshold
}
print(f"California: Selected hidden units = {parsimonious['hidden_dim']} (mean val loss: {parsimonious['mean_val']:.4f})")

# MNIST (classification)
mnist_results = hidden_unit_search("mnist", classification=True)
parsimonious, best, se_best, threshold = select_best_hidden(mnist_results, task="classification", metric="val", n_runs=n_runs, relative_rule=True)
search_results["mnist"] = {
    "results": mnist_results,
    "selected": parsimonious,
    "best": best,
    "se_best": se_best,
    "threshold": threshold
}
print(f"MNIST: Selected hidden units = {parsimonious['hidden_dim']} (mean val loss: {parsimonious['mean_val']:.4f})")

# --- Save results to file ---
os.makedirs("results", exist_ok=True)
run_id = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"results/hidden_unit_search_results_{run_id}.json"
with open(filename, "w") as f:
    json.dump(search_results, f, indent=2)

print(f"All hidden unit search results saved to {filename}")

In [ ]:
# Load results
with open("results/hidden_unit_search_results_20250930_173007.json", "r") as f:
    search_results = json.load(f)

ordered_datasets = ["sine", "iris", "stroke", "wine", "california", "mnist"]
dataset_names = [d for d in ordered_datasets if d in search_results]

rows, cols = 2, 3
fig, axes = plt.subplots(rows, cols, figsize=(5 * cols, 3.2 * rows), squeeze=False)
axes_flat = axes.flatten()

for i, ds in enumerate(dataset_names):
    ax = axes_flat[i]
    results = search_results[ds]["results"]
    hidden_sizes = [r["hidden_dim"] for r in results]
    mean_vals = [r["mean_val"] for r in results]
    std_vals = [r["std_val"] for r in results]
    ax.errorbar(hidden_sizes, mean_vals, yerr=std_vals, fmt='-o', capsize=4)
    ax.set_title(ds.capitalize())
    ax.set_xlabel("Hidden units")
    ax.set_ylabel("Mean val loss")
    ax.grid(alpha=0.3)

# Hide any unused subplot slots
for j in range(len(dataset_names), len(axes_flat)):
    axes_flat[j].set_visible(False)
fig.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

### Parameter Tuning

In [ ]:
def parameter_tuning(
    dataset_name,
    param_grid,
    hidden_dim,
    optimizer_name="sgd", 
    classification=True,
    n_runs=3,
    epochs=50,
    batch_size=32
):
    X_train, y_train = datasets[dataset_name]["train"]
    X_val, y_val = datasets[dataset_name]["val"]
    output_dim = len(np.unique(y_train)) if classification else 1
    input_dim = X_train.shape[1]

    results = []

    # Select the training function
    if optimizer_name == "sgd":
        train_fn = train_sgd
    elif optimizer_name == "scg":
        train_fn = train_scg
    elif optimizer_name == "lfrog":
        train_fn = train_lfrog
    else:
        raise ValueError(f"Unknown optimizer: {optimizer_name}")

    # Use R seed list for reproducibility
    seeds = R[:n_runs] if 'R' in globals() else list(range(n_runs))


    for params in param_grid:
        val_losses = []
        accs, f1s, mses, r2s = [], [], [], []
        times = []

        for seed in seeds:
            np.random.seed(seed)
            torch.manual_seed(seed)

            model = FeedforwardNN(
                input_dim=input_dim,
                hidden_dim=hidden_dim,
                output_dim=output_dim
            )

            if classification:
                y_train_run, y_val_run = y_train, y_val
            else:
                y_train_run = np.array(y_train).reshape(-1, 1).astype(np.float32)
                y_val_run = np.array(y_val).reshape(-1, 1).astype(np.float32)

            extra_kwargs = {k: v for k, v in params.items() if k != "hidden_dim"}

            start_time = time.time()
            if optimizer_name == "sgd":
                _, val_loss_curve = train_fn(
                    model,
                    X_train,
                    y_train_run,
                    X_val,
                    y_val_run,
                    epochs=epochs,
                    lr=extra_kwargs.get("lr", 0.01),
                    batch_size=batch_size,
                    classification=classification,
                    momentum=extra_kwargs.get("momentum", 0.0)
                )
            elif optimizer_name == "scg":
                # Convert data to torch tensors
                X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
                y_train_tensor = torch.tensor(y_train_run, dtype=torch.long if classification else torch.float32)
                X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
                y_val_tensor = torch.tensor(y_val_run, dtype=torch.long if classification else torch.float32)
                _, val_loss_curve = train_fn(
                    model,
                    X_train_tensor,
                    y_train_tensor,
                    X_val_tensor,
                    y_val_tensor,
                    max_epochs=epochs,
                    **extra_kwargs
                )
            elif optimizer_name == "lfrog":
                # Convert data to torch tensors
                X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
                y_train_tensor = torch.tensor(y_train_run, dtype=torch.long if classification else torch.float32)
                X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
                y_val_tensor = torch.tensor(y_val_run, dtype=torch.long if classification else torch.float32)
                # Choose loss_fn
                loss_fn = nn.CrossEntropyLoss() if classification else nn.MSELoss()
                _, val_loss_curve = train_fn(
                    model,
                    X_train_tensor,
                    y_train_tensor,
                    X_val_tensor,
                    y_val_tensor,
                    epochs=epochs,
                    loss_fn=loss_fn,
                    **extra_kwargs
                )
            else:
                raise ValueError(f"Unknown optimizer: {optimizer_name}")

            end_time = time.time()
            elapsed = end_time - start_time
            times.append(elapsed)
            val_losses.append(val_loss_curve[-1])

            model.eval()
            X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
            with torch.no_grad():
                outputs = model(X_val_tensor)
                if classification:
                    preds = torch.argmax(outputs, dim=1).cpu().numpy()
                    accs.append(accuracy_score(y_val, preds))
                    f1s.append(f1_score(y_val, preds, average="weighted"))
                else:
                    preds = outputs.cpu().numpy().flatten()
                    # Check for NaNs before appending metrics
                    if (
                        np.isnan(y_val_run.flatten()).any()
                        or np.isnan(preds).any()
                    ):
                        print(f"Warning: NaNs detected in validation targets or predictions for {dataset_name} (seed={seed}). Skipping metrics for this run.")
                        print("y_val_run:", y_val_run.flatten())
                        print("preds:", preds)
                    else:
                        mses.append(mean_squared_error(y_val_run.flatten(), preds))
                        r2s.append(r2_score(y_val_run.flatten(), preds))

        results.append({
            "optimizer": optimizer_name,
            "params": params,
            "mean_val_loss": float(np.mean(val_losses)),
            "std_val_loss": float(np.std(val_losses)),
            "mean_acc": float(np.mean(accs)) if accs else None,
            "std_acc": float(np.std(accs)) if accs else None,
            "mean_f1": float(np.mean(f1s)) if f1s else None,
            "std_f1": float(np.std(f1s)) if f1s else None,
            "mean_mse": float(np.mean(mses)) if mses else None,
            "std_mse": float(np.std(mses)) if mses else None,
            "mean_r2": float(np.mean(r2s)) if r2s else None,
            "std_r2": float(np.std(r2s)) if r2s else None,
            "mean_time": float(np.mean(times)),
            "std_time": float(np.std(times)),
            "all_times": [float(t) for t in times]
        })

    return results


In [ ]:
best_hidden = {
    "sine": 8,
    "iris": 4,
    "stroke": 8,
    "wine": 8,
    "california": 64,
    "mnist": 64
}

In [ ]:
# best_hidden = {ds: search_results[ds]["selected"]["hidden_dim"] for ds in search_results}



param_grid_sgd = [
    {"lr": lr, "momentum": m}
    for lr, m in product(
        [0.1, 0.05, 0.01, 0.005, 0.001, 0.0005],
        [0.0, 0.5, 0.9]
    )
]

param_grid_scg = [
    {"sigma": sigma, "lambda_init": lam}
    for sigma, lam in product(
        [1e-3, 5e-4, 1e-4, 5e-5, 1e-5],
        [1e-4, 1e-5, 1e-6, 1e-7]
    )
]

param_grid_lfrog = [
    {"dt": dt, "max_step": ms, "convergence_tol": tol}
    for dt, ms, tol in product(
        [0.5, 0.2, 0.1, 0.05, 0.01],
        [1.0, 0.5, 0.1],
        [1e-4, 1e-5, 1e-6]
    )
]

# --- Run tuning ---
all_results = {}

# # Iris (classification)
# all_results["iris"] = {
#     "sgd": parameter_tuning("iris", param_grid_sgd, hidden_dim=best_hidden["iris"], optimizer_name="sgd", classification=True),
#     "scg": parameter_tuning("iris", param_grid_scg, hidden_dim=best_hidden["iris"], optimizer_name="scg", classification=True),
#     "lfrog": parameter_tuning("iris", param_grid_lfrog, hidden_dim=best_hidden["iris"], optimizer_name="lfrog", classification=True)
# }

# Sine (regression)
all_results["sine"] = {
    "sgd": parameter_tuning("sine", param_grid_sgd, hidden_dim=best_hidden["sine"], optimizer_name="sgd", classification=False),
    "scg": parameter_tuning("sine", param_grid_scg, hidden_dim=best_hidden["sine"], optimizer_name="scg", classification=False),
    "lfrog": parameter_tuning("sine", param_grid_lfrog, hidden_dim=best_hidden["sine"], optimizer_name="lfrog", classification=False)
}

# Wine (regression)
all_results["wine"] = {
    "sgd": parameter_tuning("wine", param_grid_sgd, hidden_dim=best_hidden["wine"], optimizer_name="sgd", classification=False),
    "scg": parameter_tuning("wine", param_grid_scg, hidden_dim=best_hidden["wine"], optimizer_name="scg", classification=False),
    "lfrog": parameter_tuning("wine", param_grid_lfrog, hidden_dim=best_hidden["wine"], optimizer_name="lfrog", classification=False)
}

# Stroke (classification)
all_results["stroke"] = {
    "sgd": parameter_tuning("stroke", param_grid_sgd, hidden_dim=best_hidden["stroke"], optimizer_name="sgd", classification=True),
    "scg": parameter_tuning("stroke", param_grid_scg, hidden_dim=best_hidden["stroke"], optimizer_name="scg", classification=True),
    "lfrog": parameter_tuning("stroke", param_grid_lfrog, hidden_dim=best_hidden["stroke"], optimizer_name="lfrog", classification=True)
}

# --- Save results ---
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
out_path = f"results/param_tuning_results_{timestamp}.json"

with open(out_path, "w") as f:
    json.dump(all_results, f, indent=2)

print(f"All parameter tuning results saved to {out_path}")


In [ ]:
def select_best_one_se(results, minimization=True):
    if minimization:
        # 1. Find best (lowest mean)
        best = min(results, key=lambda x: x["mean_val_loss"])
        se_best = best["std_val_loss"]  # already averaged per seed → std across runs
        threshold = best["mean_val_loss"] + se_best

        # 2. All configs within one-SE
        candidates = [r for r in results if r["mean_val_loss"] <= threshold]

        # 3. Parsimony: choose simplest candidate (here = lowest learning rate if SGD, lowest dt if LFROG)
        # Fallback: pick the one with lowest mean_time if multiple remain
        selected = min(candidates, key=lambda x: (x["params"].get("lr", x["mean_time"])))
    else:
        # Same logic but for maximization
        best = max(results, key=lambda x: x["mean_val_loss"])
        se_best = best["std_val_loss"]
        threshold = best["mean_val_loss"] - se_best
        candidates = [r for r in results if r["mean_val_loss"] >= threshold]
        selected = min(candidates, key=lambda x: (x["params"].get("lr", x["mean_time"])))

    return {
        "best_config": best,
        "selected_config": selected,
        "threshold": threshold,
        "candidates": candidates
    }


In [ ]:
def _parsimony_key(result):
    p = result["params"]
    if "lr" in p:
        return (p["lr"], result["mean_time"])
    if "dt" in p:
        return (p["dt"], result["mean_time"])
    if "sigma" in p:
        return (p["sigma"], result["mean_time"])
    return (result["mean_time"],)

def select_best_one_se_generic(results, n_runs, minimization=True):
    if minimization:
        best = min(results, key=lambda x: x["mean_val_loss"])
        se_best = best["std_val_loss"] / np.sqrt(n_runs)
        threshold = best["mean_val_loss"] + se_best
        candidates = [r for r in results if r["mean_val_loss"] <= threshold]
    else:
        best = max(results, key=lambda x: x["mean_val_loss"])
        se_best = best["std_val_loss"] / np.sqrt(n_runs)
        threshold = best["mean_val_loss"] - se_best
        candidates = [r for r in results if r["mean_val_loss"] >= threshold]
    selected = min(candidates, key=_parsimony_key)
    return {"best_config": best,
            "selected_config": selected,
            "threshold": threshold,
            "candidates": candidates,
            "se_best": se_best}

def build_selection_summary(all_results, n_runs, minimization=True):
    rows = []
    for dataset, optim_dict in all_results.items():
        for optimizer, results in optim_dict.items():
            if not results:
                continue
            sel = select_best_one_se_generic(results, n_runs=n_runs, minimization=minimization)
            best = sel["best_config"]; chosen = sel["selected_config"]
            rows.append({
                "dataset": dataset,
                "optimizer": optimizer,
                "best_params": json.dumps(best["params"], sort_keys=True),
                "best_mean_val_loss": best["mean_val_loss"],
                "best_std_val_loss": best["std_val_loss"],
                "best_mean_time": best.get("mean_time"),
                "best_std_time": best.get("std_time"),
                "selected_params": json.dumps(chosen["params"], sort_keys=True),
                "selected_mean_val_loss": chosen["mean_val_loss"],
                "selected_std_val_loss": chosen["std_val_loss"],
                "selected_mean_time": chosen.get("mean_time"),
                "selected_std_time": chosen.get("std_time"),
                "time_ratio_selected_over_best": (
                    (chosen.get("mean_time") / best.get("mean_time"))
                    if best.get("mean_time") and chosen.get("mean_time") else None
                ),
                "threshold": sel["threshold"],
                "se_best": sel["se_best"],
                "n_candidates": len(sel["candidates"])
            })
    return pd.DataFrame(rows).sort_values(["dataset","optimizer"]).reset_index(drop=True)


In [ ]:

with open("results/param_tuning_results_20251001_185306.json", "r") as f:
    all_results = json.load(f)


sine_sgd_results = all_results["sine"]["sgd"]

selection = select_best_one_se(sine_sgd_results, minimization=True)

# Ensure results JSON loaded (reuse all_results if already in scope)
# If not loaded yet, uncomment:
# with open("results/param_tuning_results_20251001_173314.json", "r") as f:
#     all_results = json.load(f)

summary_df = build_selection_summary(all_results, n_runs=3, minimization=True)
print(summary_df)

# Save
os.makedirs("results", exist_ok=True)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
csv_path = f"results/param_tuning_selection_summary_{timestamp}.csv"
json_path = f"results/param_tuning_selection_summary_{timestamp}.json"
summary_df.to_csv(csv_path, index=False)
summary_df.to_json(json_path, orient="records", indent=2)
print(f"Saved summary to:\n  {csv_path}\n  {json_path}")


In [ ]:
def aggregate_global_best(all_results, optimizers=("sgd","scg","lfrog"), require_full_coverage=False):

    per_optimizer = {}
    for opt in optimizers:
        # Collect runs per dataset for this optimizer
        per_dataset = {
            ds: opt_dict[opt]
            for ds, opt_dict in all_results.items()
            if opt in opt_dict and opt_dict[opt]
        }
        if not per_dataset:
            continue

        # Best loss per dataset (scale-free normalization)
        best_loss = {
            ds: min(r["mean_val_loss"] for r in runs)
            for ds, runs in per_dataset.items()
        }

        metrics = defaultdict(lambda: {"losses":{}, "rel":{}, "ranks":{}, "times":{}})

        # Accumulate stats
        for ds, runs in per_dataset.items():
            sorted_runs = sorted(runs, key=lambda r: r["mean_val_loss"])
            for rank, r in enumerate(sorted_runs, start=1):
                k = tuple(sorted(r["params"].items()))
                loss = r["mean_val_loss"]
                rel_gap = (loss - best_loss[ds]) / (best_loss[ds] + 1e-12)
                m = metrics[k]
                m["losses"][ds] = loss
                m["rel"][ds] = rel_gap
                m["ranks"][ds] = rank
                m["times"][ds] = r.get("mean_time", math.nan)

        rows = []
        full_cov = len(per_dataset)
        for k, d in metrics.items():
            cov = len(d["losses"])
            if require_full_coverage and cov < full_cov:
                continue
            rels = list(d["rel"].values())
            ranks = list(d["ranks"].values())
            times = list(d["times"].values())
            rows.append({
                "optimizer": opt,
                "params": dict(k),
                "datasets_covered": cov,
                "mean_relative_gap": float(np.mean(rels)),
                "worst_relative_gap": float(np.max(rels)),
                "avg_rank": float(np.mean(ranks)),
                "mean_time_avg": float(np.nanmean(times)),
            })

        if not rows:
            continue

        df = pd.DataFrame(rows).sort_values(
            ["mean_relative_gap","worst_relative_gap","avg_rank","mean_time_avg"]
        ).reset_index(drop=True)

        per_optimizer[opt] = {
            "candidates": df.to_dict(orient="records"),
            "selected": df.iloc[0].to_dict()
        }

    return per_optimizer

global_hparam_choices = aggregate_global_best(all_results, require_full_coverage=False)

print("Global hyperparameter choices:")
for opt, info in global_hparam_choices.items():
    sel = info["selected"]
    print(f"{opt}: {sel['params']} "
          f"mean_rel_gap={sel['mean_relative_gap']:.3e} "
          f"worst_rel_gap={sel['worst_relative_gap']:.3e} "
          f"avg_rank={sel['avg_rank']:.2f}")

os.makedirs("results", exist_ok=True)
with open("results/global_hparam_choices.json","w") as f:
    json.dump(global_hparam_choices, f, indent=2)

# Optional: flat CSV of selected
selected_rows = [
    {
        "optimizer": opt,
        **info["selected"]["params"],
        **{k: v for k, v in info["selected"].items() if k not in ("params","optimizer")}
    }
    for opt, info in global_hparam_choices.items()
]
pd.DataFrame(selected_rows).to_csv("results/global_hparam_choices_selected.csv", index=False)

### Sanity checks

In [ ]:
sgd = {'lr': 0.01, 'momentum': 0.9} 
scg = {'lambda_init': 1e-07, 'sigma': 0.0005} 
lfrog = {'convergence_tol': 0.0001, 'dt': 0.2, 'max_step': 0.5} 

In [ ]:

def run_best_param_sanity_checks(
    global_hparam_choices,
    best_hidden,
    datasets_dict,
    max_epochs_sgd=100,
    max_epochs_scg=100,
    max_epochs_lfrog=100
):

    selected_params = {
        opt: info["selected"]["params"] if "selected" in info else info["params"]
        for opt, info in global_hparam_choices.items()
    }

    classification_flags = {
        "iris": True,
        "stroke": True,
        "mnist": True,
        "sine": False,
        "wine": False,
        "california": False
    }

    def prepare_targets(y, classification):
        if classification:
            return y  
        return np.array(y).reshape(-1, 1).astype(np.float32)

    lines = []
    print("\n===== SANITY CHECKS: Best Global Hyperparameters =====")
    for ds, parts in datasets_dict.items():
        if ds not in classification_flags:
            continue
        is_clf = classification_flags[ds]
        if ds not in best_hidden:
            print(f"[WARN] Missing hidden_dim for dataset {ds}, skipping.")
            continue

        (X_train, y_train) = parts["train"]
        (X_val, y_val)     = parts["val"]
        (X_test, y_test)   = parts["test"]

        X_train_full = np.vstack([X_train, X_val])
        y_train_full = np.concatenate([y_train, y_val])

        y_train_prepared = prepare_targets(y_train_full, is_clf)
        y_test_prepared  = prepare_targets(y_test, is_clf)

        input_dim  = X_train_full.shape[1]
        hidden_dim = best_hidden[ds]
        output_dim = (len(np.unique(y_train_full)) if is_clf else 1)

        print(f"\n--- Dataset: {ds} (classification={is_clf}) | hidden_dim={hidden_dim} ---")

        for opt in ("sgd", "scg", "lfrog"):
            if opt not in selected_params:
                continue
            params = selected_params[opt]
            print(f"\nOptimizer: {opt} | params={params}")

            model = FeedforwardNN(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)

            start = time.time()
            if opt == "sgd":
                lr = params.get("lr", 0.01)
                momentum = params.get("momentum", 0.0)
                train_losses, test_losses = train_sgd(
                    model,
                    X_train_full,
                    y_train_prepared,
                    X_test,
                    y_test_prepared,
                    epochs=max_epochs_sgd,
                    lr=lr,
                    batch_size=32,
                    classification=is_clf,
                    momentum=momentum
                )
            elif opt == "scg":
                sigma = params.get("sigma", 5e-5)
                lam   = params.get("lambda_init", 5e-7)
                X_train_t = torch.tensor(X_train_full, dtype=torch.float32)
                X_test_t  = torch.tensor(X_test, dtype=torch.float32)
                if is_clf:
                    y_train_t = torch.tensor(y_train_prepared, dtype=torch.long)
                    y_test_t  = torch.tensor(y_test_prepared, dtype=torch.long)
                else:
                    y_train_t = torch.tensor(y_train_prepared, dtype=torch.float32)
                    y_test_t  = torch.tensor(y_test_prepared, dtype=torch.float32)
                train_losses, test_losses = train_scg(
                    model,
                    X_train_t,
                    y_train_t,
                    X_test_t,
                    y_test_t,
                    max_epochs=max_epochs_scg,
                    sigma=sigma,
                    lambda_init=lam,
                    tolerance=1e-6,
                    verbose=False,
                    eval_freq=max(1, max_epochs_scg // 30)
                )
            else:  # lfrog
                dt = params.get("dt", 0.05)
                max_step = params.get("max_step", 0.1)
                conv_tol = params.get("convergence_tol", 1e-5)
                X_train_t = torch.tensor(X_train_full, dtype=torch.float32)
                X_test_t  = torch.tensor(X_test, dtype=torch.float32)
                if is_clf:
                    y_train_t = torch.tensor(y_train_prepared, dtype=torch.long)
                    y_test_t  = torch.tensor(y_test_prepared, dtype=torch.long)
                    loss_fn = nn.CrossEntropyLoss()
                else:
                    y_train_t = torch.tensor(y_train_prepared, dtype=torch.float32)
                    y_test_t  = torch.tensor(y_test_prepared, dtype=torch.float32)
                    loss_fn = nn.MSELoss()
                train_losses, test_losses = train_lfrog(
                    model=model,
                    X_train=X_train_t,
                    y_train=y_train_t,
                    X_test=X_test_t,
                    y_test=y_test_t,
                    loss_fn=loss_fn,
                    epochs=max_epochs_lfrog,
                    dt=dt,
                    max_step=max_step,
                    convergence_tol=conv_tol,
                    print_every=0,
                    early_stopping_patience=50,
                    min_improvement=1e-6
                )
            elapsed = time.time() - start

            model.eval()
            X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
            with torch.no_grad():
                outputs = model(X_test_tensor)

            if is_clf:
                preds = torch.argmax(outputs, dim=1).cpu().numpy()
                acc = accuracy_score(y_test, preds)
                f1  = f1_score(y_test, preds, average="weighted")
                print(f"Final Train Loss: {train_losses[-1]:.4f} | Test Loss: {test_losses[-1]:.4f} | Acc: {acc:.4f} | F1(w): {f1:.4f} | Time: {elapsed:.2f}s")
                lines.append({
                    "dataset": ds,
                    "optimizer": opt,
                    "params": params,
                    "train_loss": train_losses[-1],
                    "test_loss": test_losses[-1],
                    "accuracy": acc,
                    "f1_weighted": f1,
                    "time_sec": elapsed
                })
            else:
                preds = outputs.cpu().numpy().flatten()
                mse = mean_squared_error(y_test_prepared.flatten(), preds)
                rmse = mse ** 0.5
                r2 = r2_score(y_test_prepared.flatten(), preds)
                print(f"Final Train Loss: {train_losses[-1]:.4f} | Test Loss: {test_losses[-1]:.4f} | MSE: {mse:.4f} | RMSE: {rmse:.4f} | R2: {r2:.4f} | Time: {elapsed:.2f}s")
                lines.append({
                    "dataset": ds,
                    "optimizer": opt,
                    "params": params,
                    "train_loss": train_losses[-1],
                    "test_loss": test_losses[-1],
                    "mse": mse,
                    "rmse": rmse,
                    "r2": r2,
                    "time_sec": elapsed
                })

    print("\n===== SUMMARY (compact) =====")
    for rec in lines:
        if "accuracy" in rec:
            print(f"{rec['dataset']:<11} {rec['optimizer']:<6} loss(train/test)= {rec['train_loss']:.3f}/{rec['test_loss']:.3f} acc={rec['accuracy']:.3f} f1={rec['f1_weighted']:.3f}")
        else:
            print(f"{rec['dataset']:<11} {rec['optimizer']:<6} loss(train/test)= {rec['train_loss']:.3f}/{rec['test_loss']:.3f} mse={rec['mse']:.3f} r2={rec['r2']:.3f}")

if 'global_hparam_choices' not in globals():
    try:
        with open("results/global_hparam_choices.json","r") as f:
            global_hparam_choices = json.load(f)
        print("Loaded global_hparam_choices from file.")
    except FileNotFoundError:
        print("global_hparam_choices not found. Please run the aggregation cell first.")
else:
    print("Using in-memory global_hparam_choices.")

run_best_param_sanity_checks(global_hparam_choices, best_hidden, datasets)

### Final Experiments

In [ ]:
def run_final_experiments(
    datasets_dict,
    best_hidden,
    global_hparam_choices,
    n_runs=3,
    epochs_map=None,
    batch_size=32,
    out_prefix="results/final_experiments",
    classification_flags=None,
    save_models=True,
    model_dir="results/models",
    save_state_dict=True,
    save_full_model=False,
    target_delta=0.02,
    compute_time_to_target=True
):
    os.makedirs("results", exist_ok=True)
    if save_models:
        os.makedirs(model_dir, exist_ok=True)

    if classification_flags is None:
        classification_flags = {
            "iris": True,
            "stroke": True,
            "mnist": True,
            "sine": False,
            "wine": False,
            "california": False
        }

    selected_params = {
        opt: info["selected"]["params"] if "selected" in info else info["params"]
        for opt, info in global_hparam_choices.items()
    }

    if epochs_map is None:
        epochs_map = {"sgd": 100, "scg": 100, "lfrog": 100}

    seeds = R[:n_runs] if 'R' in globals() else list(range(n_runs))

    all_run_records = []
    summary_rows = []
    model_manifest = []

    def _first_time_to_target(loss_curve, time_curve, target):
        for l, t in zip(loss_curve, time_curve):
            if l <= target:
                return t
        return None

    def _prep_targets(y, classification):
        return y if classification else np.array(y).reshape(-1, 1).astype(np.float32)

    def _metrics(y_true, y_pred, classification):
        if classification:
            return {
                "accuracy": float(accuracy_score(y_true, y_pred)),
                "f1_weighted": float(f1_score(y_true, y_pred, average="weighted")),
                "confusion_matrix": confusion_matrix(y_true, y_pred).tolist()
            }
        mse = mean_squared_error(y_true, y_pred)
        mae = mean_absolute_error(y_true, y_pred)
        return {
            "mse": float(mse),
            "rmse": float(np.sqrt(mse)),
            "r2": float(r2_score(y_true, y_pred)),
            "mae": float(mae)
        }

    def _downsample_curve(curve, max_points=300, keep_ends=True):
        if max_points is None or len(curve) <= max_points:
            return [float(x) for x in curve]
        n = len(curve)
        idx = np.linspace(0, n - 1, max_points, dtype=int)
        if keep_ends:
            idx[0], idx[-1] = 0, n - 1
        return [float(curve[i]) for i in idx]

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    for ds, parts in datasets_dict.items():
        if ds not in classification_flags or ds not in best_hidden:
            continue
        is_clf = classification_flags[ds]

        (X_train, y_train) = parts["train"]
        (X_val, y_val) = parts["val"]
        (X_test, y_test) = parts["test"]

        # Merge train + val
        X_train_full = np.vstack([X_train, X_val])
        y_train_full = np.concatenate([y_train, y_val])

        input_dim = X_train_full.shape[1]
        hidden_dim = best_hidden[ds]
        output_dim = (len(np.unique(y_train_full)) if is_clf else 1)
        dataset_run_records = []

        for opt in ("sgd", "scg", "lfrog"):
            if opt not in selected_params:
                continue
            hparams = selected_params[opt]
            epoch_budget = epochs_map.get(opt, 100)

            for run_idx, seed in enumerate(seeds, start=1):
                np.random.seed(seed)
                torch.manual_seed(seed)

                model = FeedforwardNN(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, dropout_p=0.3)

                y_train_prep = _prep_targets(y_train_full, is_clf)
                y_test_prep = _prep_targets(y_test, is_clf)

                start_time = time.time()
                if opt == "sgd":
                    train_curve, test_curve, test_times = train_sgd(
                        model,
                        X_train_full,
                        y_train_prep,
                        X_test,
                        y_test_prep,
                        epochs=epoch_budget,
                        lr=hparams.get("lr", 0.01),
                        batch_size=batch_size,
                        classification=is_clf,
                        momentum=hparams.get("momentum", 0.0),
                        record_times=True,
                        early_stopping_patience=15
                    )
                elif opt == "scg":
                    X_tr_t = torch.tensor(X_train_full, dtype=torch.float32)
                    X_te_t = torch.tensor(X_test, dtype=torch.float32)
                    if is_clf:
                        y_tr_t = torch.tensor(y_train_prep, dtype=torch.long)
                        y_te_t = torch.tensor(y_test_prep, dtype=torch.long)
                    else:
                        y_tr_t = torch.tensor(y_train_prep, dtype=torch.float32)
                        y_te_t = torch.tensor(y_test_prep, dtype=torch.float32)
                    train_curve, test_curve, test_times = train_scg(
                        model,
                        X_tr_t,
                        y_tr_t,
                        X_te_t,
                        y_te_t,
                        max_epochs=epoch_budget,
                        sigma=hparams.get("sigma", 5e-5),
                        lambda_init=hparams.get("lambda_init", 5e-7),
                        tolerance=1e-6,
                        verbose=False,
                        eval_freq=max(1, epoch_budget // 30),
                        record_times=True
                    )
                else:  # lfrog
                    X_tr_t = torch.tensor(X_train_full, dtype=torch.float32)
                    X_te_t = torch.tensor(X_test, dtype=torch.float32)
                    if is_clf:
                        y_tr_t = torch.tensor(y_train_prep, dtype=torch.long)
                        y_te_t = torch.tensor(y_test_prep, dtype=torch.long)
                        loss_fn = nn.CrossEntropyLoss()
                    else:
                        y_tr_t = torch.tensor(y_train_prep, dtype=torch.float32)
                        y_te_t = torch.tensor(y_test_prep, dtype=torch.float32)
                        loss_fn = nn.MSELoss()
                    train_curve, test_curve, test_times = train_lfrog(
                        model=model,
                        X_train=X_tr_t,
                        y_train=y_tr_t,
                        X_test=X_te_t,
                        y_test=y_te_t,
                        loss_fn=loss_fn,
                        epochs=epoch_budget,
                        dt=hparams.get("dt", 0.05),
                        max_step=hparams.get("max_step", 0.1),
                        convergence_tol=hparams.get("convergence_tol", 1e-5),
                        early_stopping_patience=50,
                        min_improvement=1e-6,
                        print_every=0,
                        record_times=True
                    )
                elapsed = time.time() - start_time

                # Evaluate
                model.eval()
                with torch.no_grad():
                    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
                    outputs = model(X_test_tensor)
                    if is_clf:
                        preds = torch.argmax(outputs, dim=1).cpu().numpy()
                        metrics = _metrics(y_test, preds, True)
                    else:
                        preds = outputs.cpu().numpy().flatten()
                        metrics = _metrics(y_test_prep.flatten(), preds, False)

                # Save model artifacts
                model_path = None
                if save_models:
                    base_name = f"{ds}_{opt}_run{run_idx}_seed{seed}_{timestamp}"
                    state_path = os.path.join(model_dir, base_name + "_state.pt")
                    payload = {
                        "timestamp": timestamp,
                        "dataset": ds,
                        "optimizer": opt,
                        "run": run_idx,
                        "seed": seed,
                        "input_dim": input_dim,
                        "hidden_dim": hidden_dim,
                        "output_dim": output_dim,
                        "epoch_budget": epoch_budget,
                        "hyperparams": hparams,
                        "train_loss_curve": train_curve,
                        "test_loss_curve": test_curve,
                        "final_train_loss": float(train_curve[-1]),
                        "final_test_loss": float(test_curve[-1]),
                        "metrics": metrics
                    }
                    if save_state_dict:
                        payload["model_state_dict"] = model.state_dict()
                    torch.save(payload, state_path)
                    model_path = state_path
                    if save_full_model:
                        full_path = os.path.join(model_dir, base_name + "_full.pt")
                        torch.save(model, full_path)
                        model_manifest.append({
                            "dataset": ds,
                            "optimizer": opt,
                            "run": run_idx,
                            "seed": seed,
                            "type": "full_model",
                            "path": full_path
                        })
                    model_manifest.append({
                        "dataset": ds,
                        "optimizer": opt,
                        "run": run_idx,
                        "seed": seed,
                        "type": "state_payload",
                        "path": state_path
                    })

                param_count = sum(p.numel() for p in model.parameters())
                max_curve_points = 400
                train_curve_ds = _downsample_curve(train_curve, max_points=max_curve_points)
                test_curve_ds = _downsample_curve(test_curve, max_points=max_curve_points)

                record = {
                    "dataset": ds,
                    "optimizer": opt,
                    "run": run_idx,
                    "seed": seed,
                    "hidden_dim": hidden_dim,
                    "epoch_budget": epoch_budget,
                    "params": hparams,
                    "train_loss_curve": train_curve_ds,
                    "test_loss_curve": test_curve_ds,
                    "original_curve_len": len(train_curve),
                    "stored_curve_len": len(train_curve_ds),
                    "final_train_loss": float(train_curve[-1]),
                    "final_test_loss": float(test_curve[-1]),
                    "best_train_loss": float(min(train_curve)),
                    "best_test_loss": float(min(test_curve)),
                    "time_sec": float(elapsed),
                    "n_curve_points": len(train_curve_ds),
                    "model_path": model_path,
                    "param_count": int(param_count),
                    "test_time_curve": test_times
                }
                if compute_time_to_target:
                    record["full_test_loss_curve"] = [float(x) for x in test_curve]

                record.update(metrics)
                all_run_records.append(record)
                dataset_run_records.append(record)

        if compute_time_to_target and dataset_run_records:
            for opt in ("sgd", "scg", "lfrog"):
                opt_runs = [r for r in dataset_run_records if r["optimizer"] == opt]
                if not opt_runs:
                    continue
                L_star_opt = min(r["best_test_loss"] for r in opt_runs)
                target_loss_opt = L_star_opt * (1 + target_delta)
                for r in opt_runs:
                    loss_curve = r.get("full_test_loss_curve", r["test_loss_curve"])
                    time_curve = r["test_time_curve"]
                    t_hit = _first_time_to_target(loss_curve, time_curve, target_loss_opt)
                    r["opt_target_loss"] = target_loss_opt
                    r["opt_time_to_target_sec"] = t_hit
                    r["opt_reached_target"] = t_hit is not None

        for opt in ("sgd", "scg", "lfrog"):
            if opt not in selected_params:
                continue
            ds_opt_runs = [r for r in dataset_run_records if r["optimizer"] == opt]
            if not ds_opt_runs:
                continue
            final_tests = [r["final_test_loss"] for r in ds_opt_runs]
            summary = {
                "dataset": ds,
                "optimizer": opt,
                "hidden_dim": hidden_dim,
                "params": selected_params[opt],
                "n_runs": len(ds_opt_runs),
                "mean_final_test_loss": float(np.mean(final_tests)),
                "std_final_test_loss": float(np.std(final_tests)),
                "mean_time_sec": float(np.mean([r["time_sec"] for r in ds_opt_runs]))
            }
            if is_clf:
                summary.update({
                    "mean_accuracy": float(np.mean([r["accuracy"] for r in ds_opt_runs])),
                    "std_accuracy": float(np.std([r["accuracy"] for r in ds_opt_runs])),
                    "mean_f1_weighted": float(np.mean([r["f1_weighted"] for r in ds_opt_runs])),
                    "std_f1_weighted": float(np.std([r["f1_weighted"] for r in ds_opt_runs]))
                })
            else:
                summary.update({
                    "mean_mse": float(np.mean([r["mse"] for r in ds_opt_runs])),
                    "std_mse": float(np.std([r["mse"] for r in ds_opt_runs])),
                    "mean_r2": float(np.mean([r["r2"] for r in ds_opt_runs])),
                    "std_r2": float(np.std([r["r2"] for r in ds_opt_runs]))
                })

            if compute_time_to_target and any("opt_time_to_target_sec" in r for r in ds_opt_runs):
                hits = [r["opt_time_to_target_sec"] for r in ds_opt_runs if r.get("opt_reached_target")]
                summary.update({
                    "opt_target_loss": ds_opt_runs[0].get("opt_target_loss"),
                    "target_delta": target_delta,
                    "mean_time_to_target_sec": float(np.mean(hits)) if hits else None,
                    "std_time_to_target_sec": float(np.std(hits)) if hits else None,
                    "n_reached_target": int(sum(r.get("opt_reached_target", False) for r in ds_opt_runs))
                })
            summary_rows.append(summary)

    runs_json = f"{out_prefix}_{timestamp}.json"
    summary_csv = f"{out_prefix}_summary_{timestamp}.csv"
    with open(runs_json, "w") as f:
        json.dump(all_run_records, f, indent=2)
    pd.DataFrame(summary_rows).to_csv(summary_csv, index=False)

    # Save model manifest
    manifest_path = None
    if save_models:
        manifest_path = os.path.join(model_dir, f"model_manifest_{timestamp}.json")
        with open(manifest_path, "w") as f:
            json.dump(model_manifest, f, indent=2)

    print(f"Saved runs JSON   : {runs_json}")
    print(f"Saved summary CSV : {summary_csv}")
    if manifest_path:
        print(f"Saved model manifest: {manifest_path}")
    return all_run_records, pd.DataFrame(summary_rows), manifest_path

In [ ]:
final_runs, final_summary, manifest = run_final_experiments(
    datasets_dict=datasets,
    best_hidden=best_hidden,
    global_hparam_choices=global_hparam_choices,
    n_runs=3,
    epochs_map={"sgd":120,"scg":150,"lfrog":150},
    save_models=True,
    save_full_model=False
)

### Visualisation and Results

In [ ]:
final_summary = pd.read_csv("results/final_experiments_summary_20251002_201217.csv")  
# Classification subset
clf = final_summary[final_summary.dataset.isin(["iris","stroke","mnist"])]
clf_table = (clf
    .pivot(index="dataset", columns="optimizer",
           values=["mean_accuracy","mean_f1_weighted","mean_time_sec"])
    .round(4))
print(clf_table)

# Regression subset
reg = final_summary[~final_summary.dataset.isin(["iris","stroke","mnist"])]
reg_table = (reg
    .pivot(index="dataset", columns="optimizer",
           values=["mean_mse","mean_r2","mean_time_sec"])
    .round(4))
print(reg_table)

In [ ]:

with open("results/final_experiments_20251002_113644.json") as f:
    runs = json.load(f)

def plot_convergence(dataset, optimizer, curve_key="test_loss_curve"):
    ds_runs = [r for r in runs if r["dataset"]==dataset and r["optimizer"]==optimizer]
    curves = [r[curve_key] for r in ds_runs]
    max_len = max(len(c) for c in curves)
    # Pad with NaN then compute mean ignoring NaN
    arr = np.full((len(curves), max_len), np.nan)
    for i,c in enumerate(curves):
        arr[i,:len(c)] = c
    mean = np.nanmean(arr, axis=0)
    std  = np.nanstd(arr, axis=0)
    x = np.arange(len(mean))
    plt.fill_between(x, mean-std, mean+std, alpha=0.2)
    plt.plot(x, mean, label=f"{optimizer}")
    plt.xlabel("Evaluation step")
    plt.ylabel("Test loss")
    plt.title(f"{dataset} convergence")

for ds in sorted(set(r["dataset"] for r in runs)):
    plt.figure(figsize=(6,4))
    for opt in ["sgd","scg","lfrog"]:
        if any(r["dataset"]==ds and r["optimizer"]==opt for r in runs):
            plot_convergence(ds,opt)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
df = pd.DataFrame(runs)
plt.figure(figsize=(7,4))
sns.boxplot(data=df, x="dataset", y="final_test_loss", hue="optimizer")
plt.xticks(rotation=45)
plt.title("Final test loss distribution")
plt.tight_layout()
plt.show()

In [ ]:

df = pd.DataFrame(runs)

if "final_train_loss" not in df.columns:
    raise ValueError("final_train_loss missing from runs records; ensure run_final_experiments saved it.")

df["generalization_gap"] = df["final_test_loss"] - df["final_train_loss"]
df["gap_ratio"] = df["final_test_loss"] / (df["final_train_loss"] + 1e-12)

plt.figure(figsize=(8,4))
sns.boxplot(data=df, x="dataset", y="final_test_loss", hue="optimizer")
plt.xticks(rotation=45)
plt.title("Final test loss distribution")
plt.tight_layout()
plt.show()

plt.figure(figsize=(8,4))
sns.boxplot(data=df, x="dataset", y="final_train_loss", hue="optimizer")
plt.xticks(rotation=45)
plt.title("Final train loss distribution")
plt.tight_layout()
plt.show()

plt.figure(figsize=(8,4))
sns.boxplot(data=df, x="dataset", y="generalization_gap", hue="optimizer")
plt.axhline(0, color="k", lw=1, ls="--")
plt.xticks(rotation=45)
plt.title("Generalization gap (test - train)")
plt.tight_layout()
plt.show()

plt.figure(figsize=(8,4))
sns.boxplot(data=df, x="dataset", y="generalization_gap", hue="optimizer", dodge=True, showcaps=False, boxprops={'alpha':0.6})
sns.stripplot(data=df, x="dataset", y="generalization_gap", hue="optimizer", dodge=True, marker="o",
              alpha=0.6, linewidth=0.5, edgecolor="gray")
plt.xticks(rotation=45)
plt.title("Generalization gap with individual runs")
plt.tight_layout()
plt.show()

# Summarize gaps
gap_summary = (df.groupby(["dataset","optimizer"])
                 .agg(mean_gap=("generalization_gap","mean"),
                      std_gap=("generalization_gap","std"),
                      mean_test=("final_test_loss","mean"),
                      mean_train=("final_train_loss","mean"))
                 .reset_index()
              )
print(gap_summary.sort_values("mean_gap"))

In [ ]:

def plot_convergence(
    dataset,
    optimizer,
    curve_key="test_loss_curve",
    burn_in=1,
    logy=False,
    normalize=False,
    smooth_window=0,
    cap_percentile=None
):
    ds_runs = [r for r in runs if r["dataset"]==dataset and r["optimizer"]==optimizer]
    curves = [r[curve_key] for r in ds_runs]

    if burn_in > 0:
        curves = [c[burn_in:] for c in curves if len(c) > burn_in]

    # Normalize by first value
    if normalize:
        norm_curves = []
        for c in curves:
            if c and c[0] != 0:
                norm_curves.append([x / c[0] for x in c])
        curves = norm_curves if norm_curves else curves


    if smooth_window and smooth_window > 1:
        def smooth(c, w):
            if len(c) < w:
                return c
            return [
                np.mean(c[max(0,i-w+1):i+1])
                for i in range(len(c))
            ]
        curves = [smooth(c, smooth_window) for c in curves]


    if not curves:
        return
    max_len = max(len(c) for c in curves)
    arr = np.full((len(curves), max_len), np.nan)
    for i,c in enumerate(curves):
        arr[i,:len(c)] = c
    mean = np.nanmean(arr, axis=0)
    std  = np.nanstd(arr, axis=0)


    if cap_percentile:
        cap_val = np.nanpercentile(mean, cap_percentile)
        mean = np.minimum(mean, cap_val)
        std = np.minimum(std, cap_val)

    x = np.arange(len(mean))
    plt.fill_between(x, mean-std, mean+std, alpha=0.2)
    plt.plot(x, mean, label=f"{optimizer}")
    plt.xlabel("Evaluation step")
    plt.ylabel("Normalized loss" if normalize else "Test loss")
    if logy:
        plt.yscale("log")
    plt.title(f"{dataset} convergence")

# Example usage with better readability
for ds in sorted(set(r["dataset"] for r in runs)):
    plt.figure(figsize=(6,4))
    for opt in ["sgd","scg","lfrog"]:
        if any(r["dataset"]==ds and r["optimizer"]==opt for r in runs):
            plot_convergence(
                ds, opt,
                burn_in=1,     
                logy=True,        
                normalize=False,  
                smooth_window=3,  
                cap_percentile=99 
            )
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:

runs_df = pd.DataFrame(runs)


time_summary = (runs_df
    .groupby(["dataset","optimizer"])
    .agg(mean_time=("time_sec","mean"),
         std_time=("time_sec","std"),
         runs=("time_sec","count"))
    .reset_index())

plt.figure(figsize=(9,4))
sns.barplot(data=time_summary, x="dataset", y="mean_time", hue="optimizer", capsize=0.15)
for i,row in time_summary.iterrows():
    plt.text(i//3 + (i%3)*0.0, 0, "", alpha=0)  # keep structure robust
plt.ylabel("Mean training time (s)")
plt.title("Wall-clock training time (mean ± variability)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 2. Relative time vs fastest optimizer per dataset
fastest = time_summary.groupby("dataset")["mean_time"].min().rename("fastest_time")
rel = time_summary.merge(fastest, on="dataset")
rel["time_ratio"] = rel["mean_time"] / rel["fastest_time"]

plt.figure(figsize=(9,4))
sns.barplot(data=rel, x="dataset", y="time_ratio", hue="optimizer")
plt.ylabel("Time / Fastest")
plt.title("Relative training time (lower is faster)")
plt.axhline(1.0, color="k", ls="--", lw=1)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 3. Loss vs wall-clock time curves (aggregated) for each dataset
def plot_loss_vs_time(dataset, curve_key="test_loss_curve", time_key="test_time_curve", logy=False):
    ds = [r for r in runs if r["dataset"] == dataset]
    if not ds:
        return
    plt.figure(figsize=(6,4))
    for opt in ["sgd","scg","lfrog"]:
        opt_runs = [r for r in ds if r["optimizer"] == opt]
        if not opt_runs:
            continue
        # Interpolate each run onto a common time grid
        # Build union grid capped to min(max_time_across_runs, percentile)
        max_end = np.median([r[time_key][-1] for r in opt_runs if len(r[time_key])])
        grid = np.linspace(0, max_end, 200)
        interp_curves = []
        for r in opt_runs:
            t = np.array(r[time_key])
            L = np.array(r[curve_key])
            if len(t) < 2:
                continue
            interp = np.interp(grid, t, L)
            interp_curves.append(interp)
        if not interp_curves:
            continue
        arr = np.vstack(interp_curves)
        mean = arr.mean(0)
        std  = arr.std(0)
        plt.fill_between(grid, mean-std, mean+std, alpha=0.15)
        plt.plot(grid, mean, label=opt)
    plt.xlabel("Time (s)")
    plt.ylabel("Test loss")
    if logy:
        plt.yscale("log")
    plt.title(f"{dataset} loss vs time")
    plt.legend()
    plt.tight_layout()
    plt.show()

for ds in sorted(runs_df.dataset.unique()):
    plot_loss_vs_time(ds, logy=True)

# 4. Speed-to-quality metrics
def compute_speed_metrics(r, percentages=(0.5,0.9), curve_key="test_loss_curve", time_key="test_time_curve"):
    L = np.array(r[curve_key])
    T = np.array(r[time_key])
    if len(L) == 0 or len(T) == 0:
        return {}
    L0 = L[0]
    Lmin = L.min()
    out = {}
    span = L0 - Lmin
    for p in percentages:
        target = L0 - p * span

        idx = np.where(L <= target)[0]
        out[f"time_to_{int(p*100)}pct_reduction"] = float(T[idx[0]]) if len(idx) else None
    return out

speed_rows = []
for r in runs:
    metrics = compute_speed_metrics(r)
    speed_rows.append({
        "dataset": r["dataset"],
        "optimizer": r["optimizer"],
        "run": r["run"],
        **metrics
    })
speed_df = pd.DataFrame(speed_rows)

speed_summary = (speed_df
    .groupby(["dataset","optimizer"])
    .agg(**{
        "mean_t50": ("time_to_50pct_reduction","mean"),
        "std_t50": ("time_to_50pct_reduction","std"),
        "mean_t90": ("time_to_90pct_reduction","mean"),
        "std_t90": ("time_to_90pct_reduction","std"),
        "n": ("run","count")
    })
    .reset_index())

print("Speed summary (seconds):")
print(speed_summary)

# 5. Plot time-to-90% reduction (where available)
plt.figure(figsize=(9,4))
sns.barplot(data=speed_summary, x="dataset", y="mean_t90", hue="optimizer")
plt.ylabel("Time to 90% loss reduction (s)")
plt.title("Convergence speed (lower is better)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 6. Merge into final summary export (optional)
merged_summary = final_summary.merge(
    time_summary[["dataset","optimizer","mean_time","std_time"]],
    on=["dataset","optimizer"],
    how="left"
).merge(
    speed_summary[["dataset","optimizer","mean_t50","mean_t90"]],
    on=["dataset","optimizer"],
    how="left"
)

merged_summary.to_csv("results/final_with_time_metrics.csv", index=False)
print("Saved extended summary: results/final_with_time_metrics.csv")

In [ ]:
for ds in sorted(df.dataset.unique()):
    subset = df[df.dataset == ds]
    plt.figure(figsize=(5,4))
    sns.boxplot(data=subset, x="optimizer", y="final_test_loss")
    sns.stripplot(data=subset, x="optimizer", y="final_test_loss", color="k", alpha=0.5, dodge=True)
    plt.title(f"{ds} - Final test loss distribution")
    plt.ylabel("Final test loss")
    plt.xlabel("Optimizer")
    plt.tight_layout()
    plt.show()


In [ ]:

df["generalization_gap"] = df["final_test_loss"] - df["final_train_loss"]
df["gap_ratio"] = df["final_test_loss"] / (df["final_train_loss"] + 1e-12)

for ds in sorted(df.dataset.unique()):
    subset = df[df.dataset == ds]
    plt.figure(figsize=(5,4))
    sns.boxplot(data=subset, x="optimizer", y="generalization_gap")
    sns.stripplot(data=subset, x="optimizer", y="generalization_gap", color="k", alpha=0.5, dodge=True)
    plt.axhline(0, color="k", lw=1, ls="--")
    plt.title(f"{ds} - Generalization gap (test - train)")
    plt.ylabel("Generalization gap")
    plt.xlabel("Optimizer")
    plt.tight_layout()
    plt.show()

In [ ]:

if 'final_summary' not in globals():
    final_summary = pd.read_csv(sorted(
        [p for p in os.listdir("results") if p.startswith("final_experiments_summary_")],
        reverse=True
    )[0].join("results/"))

classification_datasets = ["iris", "stroke", "mnist"]
regression_datasets = [d for d in final_summary.dataset.unique() if d not in classification_datasets]

for ds in classification_datasets:
    sub = final_summary[final_summary.dataset == ds]
    if sub.empty: 
        continue
    fig, axes = plt.subplots(1, 2, figsize=(8, 3.2), sharey=False)
    order = sorted(sub.optimizer.unique())

    # Accuracy
    sns.barplot(data=sub, x="optimizer", y="mean_accuracy", order=order, ax=axes[0], palette="tab10")
    axes[0].set_title(f"{ds} - Accuracy")
    axes[0].set_xlabel("Optimizer")
    axes[0].set_ylabel("Mean Accuracy")
    for p in axes[0].patches:
        v = p.get_height()
        axes[0].annotate(f"{v:.3f}", (p.get_x()+p.get_width()/2, v), ha="center", va="bottom", fontsize=8)

    # F1
    sns.barplot(data=sub, x="optimizer", y="mean_f1_weighted", order=order, ax=axes[1], palette="tab10")
    axes[1].set_title(f"{ds} - F1 (weighted)")
    axes[1].set_xlabel("Optimizer")
    axes[1].set_ylabel("Mean F1")
    for p in axes[1].patches:
        v = p.get_height()
        axes[1].annotate(f"{v:.3f}", (p.get_x()+p.get_width()/2, v), ha="center", va="bottom", fontsize=8)

    fig.suptitle(f"{ds} (classification)")
    fig.tight_layout()
    plt.show()


for ds in regression_datasets:
    sub = final_summary[final_summary.dataset == ds]
    if sub.empty:
        continue
    fig, axes = plt.subplots(1, 2, figsize=(8, 3.2), sharey=False)
    order = sorted(sub.optimizer.unique())

    # MSE
    sns.barplot(data=sub, x="optimizer", y="mean_mse", order=order, ax=axes[0], palette="tab10")
    axes[0].set_title(f"{ds} - MSE")
    axes[0].set_xlabel("Optimizer")
    axes[0].set_ylabel("Mean MSE")
    for p in axes[0].patches:
        v = p.get_height()
        axes[0].annotate(f"{v:.3g}", (p.get_x()+p.get_width()/2, v), ha="center", va="bottom", fontsize=8)

    # R2
    sns.barplot(data=sub, x="optimizer", y="mean_r2", order=order, ax=axes[1], palette="tab10")
    axes[1].set_title(f"{ds} - R²")
    axes[1].set_xlabel("Optimizer")
    axes[1].set_ylabel("Mean R²")
    for p in axes[1].patches:
        v = p.get_height()
        axes[1].annotate(f"{v:.3f}", (p.get_x()+p.get_width()/2, v), ha="center", va="bottom", fontsize=8)
    fig.tight_layout()
    plt.show()

# OPTIONAL: Combined overview (multi-panel)
all_class_sub = final_summary[final_summary.dataset.isin(classification_datasets)]
all_reg_sub = final_summary[final_summary.dataset.isin(regression_datasets)]

In [ ]:
if 'final_summary' not in globals():

    latest = sorted(
        [p for p in os.listdir("results") if p.startswith("final_experiments_summary_")],
        reverse=True
    )[0]
    final_summary = pd.read_csv(os.path.join("results", latest))

classification_datasets = ["iris", "stroke", "mnist"]
clf_df = final_summary[final_summary.dataset.isin(classification_datasets)].copy()


pivot_table = (clf_df
               .pivot_table(index="dataset",
                            columns="optimizer",
                            values=["mean_accuracy","mean_f1_weighted"])
               .round(4)
              )

print("Pivot table (mean metrics):")
print(pivot_table)

In [ ]:
def plot_convergence_grid(
    runs,
    datasets_order=None,
    optimizers=("sgd","scg","lfrog"),
    curve_key="test_loss_curve",
    burn_in=1,
    smooth_window=3,
    normalize=False,
    logy=True,
    cap_percentile=99,
    ncols=3,
    figsize_per_panel=(4.0,3.2),
    sharey=False,
    annotate=True,
    palette=None
):
    if datasets_order is None:
        datasets_order = sorted({r["dataset"] for r in runs})
    if palette is None:
        palette = {"sgd":"#1f77b4","scg":"#2ca02c","lfrog":"#d62728"}

    agg = {}
    for ds in datasets_order:
        for opt in optimizers:
            ds_runs = [r for r in runs if r["dataset"]==ds and r["optimizer"]==opt and curve_key in r]
            if not ds_runs:
                continue
            curves = []
            for r in ds_runs:
                c = r[curve_key]
                if burn_in > 0 and len(c) > burn_in:
                    c = c[burn_in:]
                if normalize and c and c[0] != 0:
                    c = [x / c[0] for x in c]
                if smooth_window and smooth_window > 1 and len(c) >= smooth_window:
                    c = [np.mean(c[max(0,i-smooth_window+1):i+1]) for i in range(len(c))]
                curves.append(c)
            if not curves:
                continue
            max_len = max(len(c) for c in curves)
            arr = np.full((len(curves), max_len), np.nan)
            for i,c in enumerate(curves):
                arr[i,:len(c)] = c
            mean = np.nanmean(arr, axis=0)
            std  = np.nanstd(arr, axis=0)
            if cap_percentile:
                cap_val = np.nanpercentile(mean, cap_percentile)
                mean = np.minimum(mean, cap_val)
                std  = np.minimum(std, cap_val)
            agg[(ds,opt)] = (mean, std)

    n = len(datasets_order)
    nrows = int(np.ceil(n / ncols))
    fig_w = ncols * figsize_per_panel[0]
    fig_h = nrows * figsize_per_panel[1]
    fig, axes = plt.subplots(nrows, ncols,
                             figsize=(fig_w, fig_h),
                             sharey=sharey,
                             squeeze=False)
    axes_flat = axes.flatten()

    handles = {}
    for idx, ds in enumerate(datasets_order):
        ax = axes_flat[idx]
        for opt in optimizers:
            key = (ds,opt)
            if key not in agg:
                continue
            mean, std = agg[key]
            x = np.arange(len(mean))
            h = ax.plot(x, mean, label=opt, color=palette.get(opt, None), linewidth=1.6)[0]
            ax.fill_between(x, mean-std, mean+std, color=h.get_color(), alpha=0.18, linewidth=0)
            handles[opt] = h
            if annotate:
                ax.text(0.98, 0.02,
                        f"{opt}: {mean[-1]:.3g}",
                        transform=ax.transAxes,
                        ha="right", va="bottom",
                        fontsize=8,
                        color=h.get_color())
        ax.set_title(ds)
        ax.set_xlabel("Step")
        if idx % ncols == 0:
            ax.set_ylabel("Normalized Loss" if normalize else "Test Loss")
        if logy:
            ax.set_yscale("log")
        ax.grid(alpha=0.25, linewidth=0.5)

    # Hide unused axes
    for j in range(n, len(axes_flat)):
        axes_flat[j].set_visible(False)

    # Single legend
    fig.legend([handles[o] for o in optimizers if o in handles],
               [o.upper() for o in optimizers if o in handles],
               loc="upper center", ncol=len(handles),
               frameon=False, fontsize=10, bbox_to_anchor=(0.5, 1.02))

    fig.tight_layout(rect=[0,0,1,0.95])
    suptitle = "Convergence Curves (Mean ± Std)"
    if normalize: suptitle += " - Normalized"
    fig.suptitle(suptitle, y=0.995, fontsize=13)
    plt.show()
    return fig

plot_convergence_grid(
    runs,
    datasets_order=["iris","stroke","mnist","sine","wine","california"],  
    burn_in=1,
    smooth_window=3,
    normalize=False,   
    logy=True,
    cap_percentile=99,
    ncols=3,
    sharey=False
)

In [ ]:
regression_datasets = [d for d in final_summary.dataset.unique() if d not in ["iris","stroke","mnist"]]

reg = final_summary[final_summary.dataset.isin(regression_datasets)].copy()

reg_table = (reg
    .pivot(index="dataset", columns="optimizer", values=["mean_mse","mean_r2"])
    .round(4)
    .sort_index()
)

display(reg_table)

# Export to LaTeX
latex_reg = reg_table.to_latex(
    multicolumn=True,
    multirow=False,
    caption="Regression performance (mean MSE and R² over runs).",
    label="tab:reg_perf"
)
print(latex_reg)